In [1]:
# https://medium.com/@iryna230520/dynamic-few-shot-prompting-overcoming-context-limit-for-chatgpt-text-classification-2f70c3bd86f9

In [2]:
USE_SAME_FEW_SHOTS=True

In [3]:
temp = 0.0
num_labels = 17
model_name="gpt-3.5-turbo-0301"
# model_name="gpt-3.5-turbo-instruct"
# model_name="gpt-4-0613"

In [4]:
output_dir = "./data/output/2023.11.21-3_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_7-shots/"

final_xlsx_filename = output_dir+f"gpt_1000_texts_{num_labels}sdg_temperature_{temp}_model_name_{model_name}.xlsx"
print(final_xlsx_filename)

classification_report_filename = output_dir+f"gpt_1000_texts_{num_labels}sdg_temperature_{temp}_model_name_{model_name}_classification_report.xlsx"
print(classification_report_filename)

final_xlsx_filename_corrected = output_dir+f"gpt_1000_texts_{num_labels}sdg_temperature_{temp}_model_name_{model_name}_corrected.xlsx"
print(final_xlsx_filename_corrected)

./data/output/2023.11.21-3_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_7-shots/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301.xlsx
./data/output/2023.11.21-3_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_7-shots/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301_classification_report.xlsx
./data/output/2023.11.21-3_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_7-shots/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301_corrected.xlsx


In [5]:
# !pip install -q openai #--progress-bar off

In [6]:
# !pip install langchain

In [7]:
import os
from IPython.display import clear_output
os.environ["OPENAI_API_KEY"] = input("INSERT YOU OPENAI API KEY:")
clear_output()

In [8]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [9]:
def perform_classification(text: str, chain: LLMChain) -> str:
    """Perform classification using LangChain model API.

    Parameters
    ----------
    text : str
        Input text.
    chain : LLMChain
        LangChain model.

    Returns
    -------
    str or None
        API Response
    """
    try:
        return chain.run(text=text)
    except Exception as e:
        print(e)
        return None
    
def convert_to_dict(row: str) -> dict:
    try:
        row = row.replace("false", "False")
        row = row.replace("true", "True")
        return ast.literal_eval(row)
    except Exception as e:
        print(e, row)
        return row

In [10]:
llm = ChatOpenAI(temperature=temp, model_name=model_name, max_tokens=500)

# Caricamento dati e preparazione prompt


In [11]:
# IN QUESTO ESPERIMENTO CONSIDERO LO STESSO DATASET UTILIZZATO IN 
# GPT_zero-shot_learning/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg.ipynb
# IN MODO CHE I DATI SIANO GLI STESSI 

data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx").drop(columns=['Unnamed: 0'])
# data.index = data.text_id
# data.text_id = data.index
# data = data.drop(columns=['Unnamed: 0', 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
data

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0


In [12]:
data.shape

(1020, 7)

In [13]:
data.groupby('sdg').count()['text']

sdg
0     60
1     60
2     60
3     60
4     60
5     60
6     60
7     60
8     60
9     60
10    60
11    60
12    60
13    60
14    60
15    60
16    60
Name: text, dtype: int64

### Estraggo i dati per i few-shot

In [14]:
# TESTI ESTRATTI NEL FEW SHOT 2023.11.21-1_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_3-shots

print("""
      text_id                   doi  \
3936     5614  10.2139/SSRN.1406042   
1456     2089  10.18356/ee52a573-en   
3974     5665  10.18356/3fe10a08-en   

                                                   text  sdg  labels_negative  \
3936  The modern definitions of the inchoate offence...   16              0.0   
1456  This refers to the manufacture, use and dispos...   12              3.0   
3974  As a result, both the trade and current accoun...    8              0.0   

      labels_positive  agreement  
3936              6.0     1.0000  
1456             29.0     0.8125  
3974              5.0     1.0000
""")


      text_id                   doi  3936     5614  10.2139/SSRN.1406042   
1456     2089  10.18356/ee52a573-en   
3974     5665  10.18356/3fe10a08-en   

                                                   text  sdg  labels_negative  3936  The modern definitions of the inchoate offence...   16              0.0   
1456  This refers to the manufacture, use and dispos...   12              3.0   
3974  As a result, both the trade and current accoun...    8              0.0   

      labels_positive  agreement  
3936              6.0     1.0000  
1456             29.0     0.8125  
3974              5.0     1.0000



In [15]:
if USE_SAME_FEW_SHOTS==True:
    data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0'])
    data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
        (data_sel_4_few_shot_examples.text_id==5614) |
        (data_sel_4_few_shot_examples.text_id==2089) |
        (data_sel_4_few_shot_examples.text_id==5665) |
        (data_sel_4_few_shot_examples.text_id==370)  |
        (data_sel_4_few_shot_examples.text_id==1422) |
        (data_sel_4_few_shot_examples.text_id==4329) |
        (data_sel_4_few_shot_examples.text_id==4929)
    ]
else:
    data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0']).sample(5)

data_sel_4_few_shot_examples

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
250,370,10.1787/9789264272392-3-en,"For example, the poorest 20% of farm household...",2,0.0,9.0,1.000000
986,1422,10.1787/9789264283299-en,"Nevertheless, the average number of physician ...",3,1.0,8.0,0.777778
1456,2089,10.18356/ee52a573-en,"This refers to the manufacture, use and dispos...",12,3.0,29.0,0.812500
3026,4329,10.18356/30e5b8d3-en,With the introduction of FSC standards in the ...,15,0.0,9.0,1.000000
3444,4929,10.18356/5e8977eb-en,"Hence, climate-economy models without explicit...",13,0.0,3.0,1.000000
3936,5614,10.2139/SSRN.1406042,The modern definitions of the inchoate offence...,16,0.0,6.0,1.000000
3974,5665,10.18356/3fe10a08-en,"As a result, both the trade and current accoun...",8,0.0,5.0,1.000000


In [16]:
few_shots = ""

for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
    input_text = "Input text: \""+row['text']+"\"\n"
    output_text = "Output: \"SDG "+str(row['sdg'])+'\"\n\n'
    
    few_shots += input_text
    few_shots += output_text
    
print(few_shots)

Input text: "For example, the poorest 20% of farm households in the Philippines tend to be small-scale producers and landless workers whose rice production does not satisfy their own needs and who therefore purchase 77% of the rice that they consume on average (Chapter 4). Regional levels of stunting among children under five years of age, for example, remain worryingly high (Figure 3). While economic growth has enabled significant declines in populations suffering from chronic poverty, for some households, it may have only altered the nature of food insecurity from permanent to transitory, rather than eliminate it entirely."
Output: "SDG 2"

Input text: "Nevertheless, the average number of physician contacts per person is not greater than the EU average. For example, home visits to patients by GPs are a regular practice and patients do not usually have to wait long to get access to GPs, although waiting times for more specialised services (e.g. mental health specialists, ophthalmologi

In [17]:
# https://www.un.org/en/exhibits/page/sdgs-17-goals-transform-world#:~:text=GOAL%201%3A%20No%20Poverty%20GOAL,10%3A%20Reduced%20Inequalities%20GOAL%2011

template = """
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The following are 7 examples of expected classification given an input text:

"""+few_shots+"""


The text to be processed is:

```
{text}
```
"""

In [18]:
prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [19]:
chain = LLMChain(llm=llm, prompt=prompt)

In [20]:
# data_sel = data_sel_4_few_shot_examples
data_sel = data
# data_sel = data[data.sdg==0].sample(10)
# data_sel = data.sample(3)
# data_sel = data.iloc[356:]

data_sel

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0


In [21]:
# data_sel.iloc[:11]

In [22]:
# text_DEBUG = data_sel.iloc[10].text
# text_DEBUG

In [23]:
# perform_classification(text_DEBUG, chain)

In [25]:
# results = data_sel.apply(lambda x: perform_classication(x["text"], chain), axis=1)
import time

cnt = 0
# cnt = 355
results = []
for index, row in data_sel.iterrows():
    result = perform_classification(row["text"], chain)
    results.append((row["text_id"], result))
    cnt+=1
    #time.sleep(1)
    print(cnt, (row["text_id"], result))
    

1 (6668, 'SDG 9')
2 (6099, 'SDG 0')
3 (6471, 'SDG 16')
4 (1786, 'SDG 4')
5 (2143, 'SDG 7')
6 (3377, 'SDG 0')
7 (5589, 'SDG 0')
8 (5431, 'SDG 8')
9 (768, 'SDG 7')
10 (2735, 'SDG 11')
11 (1306, 'SDG 0')
12 (3844, 'SDG 3')
13 (1524, 'SDG 3')
14 (4660, 'SDG 3')
15 (5077, 'SDG 0')
16 (995, 'SDG 8')
17 (2166, 'SDG 0')
18 (2512, 'SDG 16')
19 (2191, 'SDG 11')
20 (6073, 'SDG 0')
21 (1376, 'SDG 13')
22 (3889, 'SDG 3')
23 (6576, 'SDG 3')
24 (3912, 'SDG 3')
25 (1565, 'SDG 12')
26 (1033, 'SDG 16')
27 (5653, 'SDG 9')
28 (940, 'SDG 8')
29 (3613, 'SDG 0')
30 (1659, 'SDG 8')
31 (848, 'SDG 9')
32 (2157, 'SDG 15')
33 (3604, 'SDG 8')
34 (3940, 'SDG 0')
35 (6228, 'SDG 3')
36 (3148, 'SDG 0')
37 (2298, 'SDG 3')
38 (4226, 'SDG 0')
39 (2335, 'SDG 3')
40 (4832, 'SDG 3')
41 (6153, 'SDG 1')
42 (6772, 'SDG 0')
43 (6340, 'SDG 0')
44 (4505, 'SDG 3')
45 (1776, 'SDG 0')
46 (1529, 'SDG 9')
47 (1838, 'SDG 3')
48 (1427, 'SDG 15')
49 (2023, 'SDG 16')
50 (854, 'SDG 8')
51 (2541, 'SDG 3')
52 (6224, 'SDG 3')
53 (6243, 'SDG 1

405 (3427, 'Output: "SDG 6"')
406 (3790, 'SDG 6')
407 (1506, 'Output: "SDG 6"')
408 (4875, 'SDG 6')
409 (2949, 'SDG 6')
410 (2967, 'SDG 6')
411 (6342, 'SDG 6')
412 (983, 'SDG 6')
413 (3105, 'SDG 6')
414 (6438, 'SDG 6')
415 (2416, 'Output: "SDG 6"')
416 (5412, 'SDG 6')
417 (1548, 'SDG 14')
418 (616, 'SDG 6')
419 (3942, 'SDG 6')
420 (2267, 'Output: "SDG 6"')
421 (1228, 'Output: "SDG 7"')
422 (5922, 'SDG 7')
423 (3438, 'SDG 7')
424 (4989, 'SDG 7')
425 (1802, 'SDG 9')
426 (6417, 'SDG 7')
427 (3138, 'SDG 7')
428 (6251, 'SDG 13')
429 (6074, 'SDG 7')
430 (5346, 'SDG 7')
431 (6226, 'SDG 12')
432 (3417, 'SDG 7')
433 (3016, 'SDG 7')
434 (6798, 'SDG 7')
435 (4893, 'SDG 7')
436 (4955, 'SDG 7')
437 (2617, 'SDG 7')
438 (4275, 'SDG 7')
439 (5874, 'SDG 7')
440 (3068, 'SDG 7')
441 (1780, 'SDG 7')
442 (1273, 'SDG 7')
443 (4795, 'SDG 7')
444 (3256, 'SDG 7')
445 (2876, 'SDG 7')
446 (751, 'SDG 7')
447 (2893, 'SDG 7')
448 (2329, 'Output: "SDG 7"')
449 (5555, 'SDG 7')
450 (1199, 'SDG 7')
451 (5626, 'SDG 7')


793 (2228, 'SDG 13')
794 (2575, 'SDG 13')
795 (5881, 'SDG 13')
796 (2540, 'SDG 17')
797 (6671, 'SDG 13')
798 (4343, 'SDG 13')
799 (1964, 'SDG 5')
800 (1620, 'SDG 13')
801 (3379, 'Output: "SDG 13"')
802 (4843, 'SDG 13')
803 (6675, 'SDG 13')
804 (6696, 'SDG 13')
805 (5030, 'SDG 17')
806 (5047, 'SDG 13')
807 (220, 'SDG 13')
808 (1613, 'SDG 12')
809 (1863, 'SDG 7')
810 (5441, 'SDG 13')
811 (3560, 'Output: "SDG 13"')
812 (5575, 'SDG 4')
813 (3518, 'SDG 13')
814 (2094, 'SDG 13')
815 (5761, 'SDG 13')
816 (4378, 'SDG 17')
817 (4186, 'SDG 13')
818 (3475, 'SDG 13')
819 (4235, 'SDG 13')
820 (836, 'SDG 13')
821 (4776, 'SDG 13')
822 (2958, 'SDG 13')
823 (3880, 'SDG 13')
824 (1912, 'SDG 13')
825 (3738, 'SDG 13')
826 (1416, 'SDG 13')
827 (6007, 'SDG 13')
828 (3451, 'SDG 13')
829 (5275, 'SDG 13')
830 (6637, 'SDG 13')
831 (5363, 'SDG 13')
832 (116, 'SDG 13')
833 (872, 'SDG 13')
834 (6170, 'SDG 13')
835 (4524, 'SDG 13')
836 (3126, 'SDG 13')
837 (2077, 'SDG 13')
838 (2818, 'Output: "SDG 13"')
839 (92, 'S

In [26]:
results

[(6668, 'SDG 9'),
 (6099, 'SDG 0'),
 (6471, 'SDG 16'),
 (1786, 'SDG 4'),
 (2143, 'SDG 7'),
 (3377, 'SDG 0'),
 (5589, 'SDG 0'),
 (5431, 'SDG 8'),
 (768, 'SDG 7'),
 (2735, 'SDG 11'),
 (1306, 'SDG 0'),
 (3844, 'SDG 3'),
 (1524, 'SDG 3'),
 (4660, 'SDG 3'),
 (5077, 'SDG 0'),
 (995, 'SDG 8'),
 (2166, 'SDG 0'),
 (2512, 'SDG 16'),
 (2191, 'SDG 11'),
 (6073, 'SDG 0'),
 (1376, 'SDG 13'),
 (3889, 'SDG 3'),
 (6576, 'SDG 3'),
 (3912, 'SDG 3'),
 (1565, 'SDG 12'),
 (1033, 'SDG 16'),
 (5653, 'SDG 9'),
 (940, 'SDG 8'),
 (3613, 'SDG 0'),
 (1659, 'SDG 8'),
 (848, 'SDG 9'),
 (2157, 'SDG 15'),
 (3604, 'SDG 8'),
 (3940, 'SDG 0'),
 (6228, 'SDG 3'),
 (3148, 'SDG 0'),
 (2298, 'SDG 3'),
 (4226, 'SDG 0'),
 (2335, 'SDG 3'),
 (4832, 'SDG 3'),
 (6153, 'SDG 1'),
 (6772, 'SDG 0'),
 (6340, 'SDG 0'),
 (4505, 'SDG 3'),
 (1776, 'SDG 0'),
 (1529, 'SDG 9'),
 (1838, 'SDG 3'),
 (1427, 'SDG 15'),
 (2023, 'SDG 16'),
 (854, 'SDG 8'),
 (2541, 'SDG 3'),
 (6224, 'SDG 3'),
 (6243, 'SDG 16'),
 (4104, 'SDG 3'),
 (1497, 'SDG 9'),
 (20

In [27]:
# len(results)

In [28]:
# len(results_part1)

In [29]:
# results_tot = results_part1 + results

In [30]:
# len(results_tot)

In [31]:
# results = results_tot

In [32]:
# results_part1 = results.copy()

In [33]:
# results_part1

In [34]:
len(results)

1020

In [35]:
results_processed = pd.DataFrame(results, columns=['text_id', 'result'])
results_processed['label_pred'] = results_processed['result'].str.extract(r'SDG (\d+)')
results_processed

,text_id,result,label_pred
0,6668,SDG 9,9
1,6099,SDG 0,0
2,6471,SDG 16,16
3,1786,SDG 4,4
4,2143,SDG 7,7
...,...,...,...
1015,4695,"Output: ""SDG 16""",16
1016,6628,"Output: ""SDG 16""",16
1017,4563,SDG 16,16
1018,6710,SDG 16,16


In [36]:
# results_processed.to_excel("gpt3_few_3_shot_tmp.xlsx")

In [37]:
# results_processed = results.apply(convert_to_dict).apply(pd.Series).loc[:, ["motivation_for_the_answer", "detected_goals", "selected_goal"]]
# results_processed

In [38]:
# results_processed2 = results.apply(convert_to_dict).apply(pd.Series).loc[:, ["goal_label", "motivation_for_the_answer", "sdg_probability"]]
# results_processed2

In [39]:
# results_processed = pd.DataFrame(results, columns=['text_id', 'result'])
# results_processed['label_pred'] = results_processed['result'].str.extract(r'SDG (\d+)')
# results_processed

In [40]:
# final = data_sel.merge(results_processed, left_index=True, right_index=True)
final = data_sel.merge(results_processed, left_on='text_id', right_on='text_id')
# final = data_sel.merge(results_processed2, left_index=True, right_index=True)
# final = final.drop(columns='Unnamed: 0')
final

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,result,label_pred
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,SDG 9,9
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,SDG 0,0
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,SDG 16,16
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,SDG 4,4
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,SDG 7,7
...,...,...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,"Output: ""SDG 16""",16
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,"Output: ""SDG 16""",16
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,SDG 16,16
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,SDG 16,16


In [41]:
final_xlsx_filename

'./data/output/2023.11.21-3_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_7-shots/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301.xlsx'

In [42]:
final.to_excel(final_xlsx_filename)

In [43]:
final_corrected = final

In [44]:
import numpy as np
y_true = final_corrected.sdg 
y_pred = final_corrected.label_pred
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       1.00      0.25      0.40        60
           1       0.67      0.83      0.74        60
           2       0.94      0.85      0.89        60
           3       0.67      0.98      0.80        60
           4       0.87      0.97      0.91        60
           5       0.90      0.95      0.93        60
           6       0.98      0.93      0.96        60
           7       0.89      0.92      0.90        60
           8       0.53      0.68      0.60        60
           9       0.74      0.75      0.74        60
          10       0.78      0.48      0.60        60
          11       0.89      0.82      0.85        60
          12       0.84      0.85      0.84        60
          13       0.80      0.87      0.83        60
          14       0.92      0.97      0.94        60
          15       0.93      0.88      0.91        60
          16       0.88      0.83      0.85        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
classification_report_filename

'./data/output/2023.11.21-3_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_7-shots/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301_classification_report.xlsx'

In [46]:
report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_excel(classification_report_filename)

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
